In [2]:
import textdistance
import pandas as pd
import os

In [6]:
s1 = "Holmes"
s2 = "Sherlock"
s3 = "Sherlock Holmes"
s4 = "Mr. Sherlock Holmes"
s5 = "Mr. Holmes"
l1 = "Lord Habla"
l2 = "Lord Mambo"
m1 = "Miss Shuterland"
m2 = "Miss Mary Shuterland"

In [3]:
#Edit based
ham = textdistance.hamming.normalized_similarity(m1,m2)
lev = textdistance.levenshtein.normalized_similarity(m1,m2)
jaro = textdistance.jaro.normalized_similarity(m1,m2)
ja_wi = textdistance.jaro_winkler.normalized_similarity(m1,m2)
print(ham, lev, jaro, ja_wi)

0.25 0.75 0.8277777777777778 0.8966666666666667


In [4]:
ham = textdistance.Hamming(qval=2).normalized_similarity(m1,m2)
lev = textdistance.Levenshtein(qval=2).normalized_similarity(m1,m2)
jaro = textdistance.Jaro(qval=2).normalized_similarity(m1,m2)
ja_wi = textdistance.JaroWinkler(qval=2).normalized_similarity(m1,m2)
print(ham, lev, jaro, ja_wi)

0.25 0.75 0.912280701754386 0.9473684210526316


In [5]:
#Token based
jaccard = textdistance.jaccard.normalized_similarity(m1,m2)
#dice = textdistance.dice.normalized_similarity(m1,m2)
sor = textdistance.sorensen.normalized_similarity(m1,m2)
sor_dice = textdistance.sorensen_dice.normalized_similarity(m1,m2)
overlap = textdistance.overlap.normalized_similarity(m1,m2)
cos = textdistance.cosine.normalized_similarity(m1,m2)
mon_elk = textdistance.monge_elkan.normalized_similarity(m1,m2)
bag = textdistance.bag.normalized_similarity(m1,m2)
print(jaccard, sor, sor_dice, overlap, cos, mon_elk, bag)

0.75 0.8571428571428571 0.8571428571428571 1.0 0.8660254037844386 0.033333333333333326 0.75


In [6]:
jaccard = textdistance.Jaccard(qval=2).normalized_similarity(m1,m2)
#dice = textdistance.dice.normalized_similarity(m1,m2)
sor = textdistance.Sorensen(qval=2).normalized_similarity(m1,m2)
#sor_dice = textdistance.Sorensen.normalized_similarity(m1,m2)
overlap = textdistance.Overlap(qval=2).normalized_similarity(m1,m2)
cos = textdistance.Cosine(qval=2).normalized_similarity(m1,m2)
mon_elk = textdistance.MongeElkan(qval=2).normalized_similarity(m1,m2)
bag = textdistance.Bag(qval=2).normalized_similarity(m1,m2)
print(jaccard, sor, overlap, cos, mon_elk, bag)

0.7368421052631579 0.8484848484848485 1.0 0.8583950752789522 0.0714285714285714 0.75


In [14]:
overlap1 = textdistance.Overlap(qval=2).normalized_similarity('London Library', 'East London')
overlap2 = textdistance.Overlap(qval=2).normalized_similarity('East London', 'London')
print(overlap1, overlap2)

0.6 1.0


In [7]:
#Sequence based
lon_seq = textdistance.lcsseq.normalized_similarity(m1,m2)
lon_str = textdistance.lcsstr.normalized_similarity(m1,m2)
print(lon_seq, lon_str)

0.75 0.55


In [3]:
#get all different characters and locations
FOLDER_PATH = os.path.join('..','output_parsed','ner')
characters = []
locations = []
pronouns = ['his','he','she','her','him', 'hers', 'you', 'i', 'me', 'my', 'your']
for f in os.listdir(FOLDER_PATH):
    file = os.path.join(FOLDER_PATH, f)
    df =  pd.read_csv(file, sep=",",engine="python")
    
    for index, row in df.iterrows():
        if (row['ner'] == "PERSON") and (row['text'] not in characters) and  not (any(p.lower() == row['text'].lower() for p in pronouns)):
            characters.append(row['text'])
        if (row['ner'] == "LOCATION") and (row['text'] not in locations):
            locations.append(row['text'])


In [21]:
#find similar location names
data = []
for i in range(len(locations)):
    for j in range(i+1,len(locations)):
        overlap = textdistance.Overlap(qval=2).normalized_similarity(locations[i],locations[j])
        cosine = textdistance.Cosine(qval=2).normalized_similarity(locations[i],locations[j])
        ja_wi = textdistance.JaroWinkler(qval=2).normalized_similarity(locations[i],locations[j])
        sim = cosine + ja_wi
        if sim > 1.6:
            data.append([locations[i], locations[j], sim])
            
df = pd.DataFrame(data, columns = ["location1", "location2", "similarity"])

In [22]:
df

,location1,location2,similarity
0,Holland,Holland Grove,1.607107
1,London,London Road,1.607107
2,England,New England,1.641263
3,Camberwell,Camberwell Road,1.730355
4,Fenchurch Street,Church Street,1.642619
...,...,...,...
86,Andaman Islands,Andaman Isles,1.681041
87,Vauxhall Bridge Road,Vauxhall Bridge,1.805763
88,Birlstone,Birlstone Manor,1.670215
89,Birlstone,Birlstone Ridge,1.670215


In [37]:
data = []
for i in range(len(locations)):
    matches = []
    for j in range(i+1,len(locations)):
        #overlap = textdistance.Overlap(qval=2).normalized_similarity(locations[i],locations[j])
        cosine = textdistance.Cosine(qval=2).normalized_similarity(locations[i],locations[j])
        ja_wi = textdistance.JaroWinkler(qval=2).normalized_similarity(locations[i],locations[j])
        sim = cosine + ja_wi
        if sim > 1.6:
            matches.append(locations[j])
    if matches:
        matches.append(locations[i])
        norm = max(matches, key = len)
        matches = ' / '.join(matches)
        data.append([matches,norm])
            
df_loc = pd.DataFrame(data, columns = ["locations", "normalizedLocation"])

In [42]:
           
data = []
for i in range(len(characters)):
    matches = []
    for j in range(i+1,len(characters)):
        sim = textdistance.Overlap(qval=2).normalized_similarity(characters[i],characters[j])
        if sim > 0.8:
            matches.append(characters[j])
    if matches:
        matches.append(characters[i])
        norm = max(matches, key = len)
        matches = ' / '.join(matches)
        data.append([matches,norm])
            
df_char = pd.DataFrame(data, columns = ["characters", "normalizedCharacter"])            

In [44]:
df_char
df_char.to_csv('normCharacters.csv')

In [77]:
#no need for chracter names from quotes because there is no new form

filepath= '../output_parsed/novel_a_study_in_scarlet/quotes/'
for f in os.listdir(filepath):
    file = os.path.join(filepath, f)
    df =  pd.read_csv(file, sep=",",engine="python")
    for index, row in df.iterrows():
        if (row['speaker'] not in characters) and not (any(p.lower() == row['speaker'].lower() for p in pronouns)) and not (row['speaker'] == "Unknown"):
            characters.append(row['speaker'])
        if (row['canonicalSpeaker'] not in characters) and not (any(p.lower() == row['canonicalSpeaker'].lower() for p in pronouns)) and not (row['canonicalSpeaker'] == "Unknown"):
            characters.append(row['canonicalSpeaker'])
        
len(characters)  

1543